Packages:

- [NLopt](https://github.com/JuliaOpt/NLopt.jl) Powell's derivative-free algorithms
- [Optim](https://github.com/JuliaNLSolvers/Optim.jl) Gradient-based algorithms, autodifferentiation support

In [10]:
using NLopt
using Optim
using Printf

# Nonlinear unconstrained optimization

Example: projective measurements on the singlet state $\left| \Psi^- \right> = \left| 01 - 10 \right>/\sqrt{2} \in \mathcal{H}_\text{A} \otimes (\mathcal{H}_\text{B})$.

A for Alice and B for Bob.

We parameterize local measurements by qubit states.

$$ \left| \vec{\phi} \right> = \begin{pmatrix} \cos \phi_1 \\ e^{i \phi_2} \sin \phi_1 \end{pmatrix}, \qquad \left| \vec{\phi} \right>_\perp = \begin{pmatrix} -\sin \phi_1 \\ e^{i \phi_2} \cos \phi_1 \end{pmatrix}  $$

There are two measurement settings indexed by $s=1,2$ on $\mathcal{H}_\text{A}$, corresponding to two projectors $\{\left| \alpha_1 \right>, \left| \alpha_1 \right>_\perp\}$ and $\{\left| \alpha_2 \right>, \left| \alpha_2 \right>_\perp \}$, and two measurement settings indexed by $t=1,2$ on $\mathcal{H}_\text{B}$, corresponding to $\{\left| \beta_1 \right>, \left| \beta_1 \right>_\perp\}$ and $\{\left| \beta_2 \right>, \left| \beta_2 \right>_\perp\}$.

The measurement outcomes are 1-based (Julia, Matlab), so that $a=1$ corresponds to $\left | \alpha_s \right >$ and $a=2$ to $\left | \alpha_s \right >_\perp$. The same for Bob: for $b=1$ corresponds to $\left | \beta_t \right>$ and $b=2$ to $\left| \beta_t \right>_\perp$

Then we consider the joint, conditional probability distribution $$ P_\text{AB|ST}(a,b|s,t) $$. We compute using Born's rule:

$$ P_\text{AB|ST}(1,1|s,t) = \left( \left< \alpha_s \right | \otimes \left< \beta_t \right | \right ) \left | \Psi^- \right > $$
$$ P_\text{AB|ST}(2,1|s,t) = \left( \left< \alpha_s \right |_\perp \otimes \left< \beta_t \right | \right ) \left | \Psi^- \right > $$
$$ P_\text{AB|ST}(1,2|s,t) = \left( \left< \alpha_s \right | \otimes \left< \beta_t \right |_\perp \right ) \left | \Psi^- \right > $$
$$ P_\text{AB|ST}(2,2|s,t) = \left( \left< \alpha_s \right |_\perp \otimes \left< \beta_t \right |_\perp \right ) \left | \Psi^- \right > $$

We want to maximize the CHSH expression:

$$ C = \sum_{abst} (-1)^{(a-1)+(b-1)+(s-1)(t-1)} P_\text{AB|ST}(a,b|s,t) $$

In [11]:
function singlet_proj_prob(x)
# Computes the joint probability distribution given by two projective measurements on each subsystem of a singlet state
    P = zeros(eltype(x)::Type, (2, 2, 2, 2));
    A = zeros(Complex{eltype(x)::Type}, (2, 2, 2));
    B = zeros(Complex{eltype(x)::Type}, (2, 2, 2));
    A[:,1,1] = [ cos(x[1])
                 sin(x[1])];
    A[:,2,1] = [-sin(x[1])
                 cos(x[1])];
    A[:,1,2] = [ cos(x[2])
                 sin(x[2])];
    A[:,2,2] = [-sin(x[2])
                 cos(x[2])];
    B[:,1,1] = [ cos(x[3])
                 sin(x[3])];
    B[:,2,1] = [-sin(x[3])
                 cos(x[3])];
    B[:,1,2] = [ cos(x[4])
                 sin(x[4])];
    B[:,2,2] = [-sin(x[4])
                 cos(x[4])];
    for s = 1:2
        for t = 1:2
            for a = 1:2
                for b = 1:2
                    ov = kron(A[:,a,s], B[:,b,t])' * [0; 1; -1; 0]/sqrt(2);
                    P[a,b,s,t] = real(conj(ov)*ov);
                end
            end
        end
    end
    return P
end

singlet_proj_prob (generic function with 1 method)

In [12]:
function chsh(P)
# Computes the CHSH expression from a joint probability distribution
    C = 0;
    for s = 1:2
        for t = 1:2
            for a = 1:2
                for b = 1:2
                    C = C + (-1)^((a-1)+(b-1)+(s-1)*(t-1)) * P[a,b,s,t];
                end
            end
        end
    end
    return C
end

chsh (generic function with 1 method)

In [13]:
using Optim
# Set up objective function, minus sign for maximization
f(x) = -chsh(singlet_proj_prob(x))
# Random initial point
x0 = rand(Float64, 4)*2*pi
verbose = true;

In [14]:
# Derivative-free, and possibly not differentiable: Nelder-Mead recommended
res = Optim.optimize(f, x0, Optim.NelderMead(), Optim.Options(f_tol = 1e-8, show_trace = verbose, show_every = 10))
@printf("\n\nMaximum: %f in %d iterations\n", -Optim.minimum(res), Optim.iterations(res))
@printf("Diff: %0.1E\n", abs(Optim.minimum(res) + 2*sqrt(2)))

Iter     Function value    √(Σ(yᵢ-ȳ)²)/n 
------   --------------    --------------
     0    -2.052920e+00     1.712428e+00
 * time: 4.601478576660156e-5
    10    -2.061950e+00     9.186981e-01
 * time: 0.0004200935363769531
    20    -2.448260e+00     1.259492e-01
 * time: 0.0007059574127197266
    30    -2.785470e+00     1.507191e-02
 * time: 0.0009741783142089844
    40    -2.822231e+00     1.482387e-03
 * time: 0.0012400150299072266
    50    -2.827533e+00     4.556943e-04
 * time: 0.0015010833740234375
    60    -2.828306e+00     3.392680e-05
 * time: 0.0017590522766113281
    70    -2.828408e+00     1.394863e-05
 * time: 0.002020120620727539
    80    -2.828426e+00     1.232248e-06
 * time: 0.002290964126586914
    90    -2.828427e+00     1.385119e-07
 * time: 0.002565145492553711


Maximum: 2.828427 in 99 iterations
Diff: 7.6E-09


In [15]:
# Derivative-free but differentiable: Powell's BOBYQA building a quadratic model

# Uses NLopt, so different calling convention.
# Warning: Powell's method becomes interesting for medium scale problems, here not much difference with Nelder-Mead

function vis(x::Vector, grad::Vector)
# we fake a "grad" argument to make NLopt happy
   val = -chsh(singlet_proj_prob(x))
   return val
end

opt = Opt(:LN_BOBYQA, 4)
opt.lower_bounds = zeros(4)*1.0
opt.upper_bounds = ones(4)*2*pi*1.0
opt.min_objective = vis
opt.ftol_rel = 1e-8
(minf, minx, ret) = NLopt.optimize(opt, x0)
@printf("\n\nMaximum: %f in %d iterations\n", -minf, opt.numevals)
@printf("Diff: %0.1E\n", abs(minf + 2*sqrt(2)))



Maximum: 2.828427 in 70 iterations
Diff: 8.8E-08


In [16]:
# Using information in provided Hessian, here using autodifferentiation
res = Optim.optimize(f, x0, Optim.Newton(), Optim.Options(f_tol = 1e-8, show_trace = verbose, show_every = 10); autodiff = :forward)
@printf("\n\nMaximum: %f in %d iterations\n", -Optim.minimum(res), Optim.iterations(res))
@printf("Diff: %0.1E\n", abs(Optim.minimum(res) + 2*sqrt(2)))

Iter     Function value   Gradient norm 
     0     1.308356e+00     3.189312e+00
 * time: 4.00543212890625e-5


Maximum: 2.828427 in 7 iterations
Diff: 1.3E-15


In [17]:
# Only gradients provided: BSGS recommended (medium scale), Hessian is approximated, gradients from autodifferentation
res = Optim.optimize(f, x0, Optim.BFGS(), Optim.Options(f_tol = 1e-8, show_trace = verbose, show_every = 10); autodiff = :forward)
@printf("\n\nMaximum: %f in %d iterations\n", -Optim.minimum(res), Optim.iterations(res))
@printf("Diff: %0.1E\n", abs(Optim.minimum(res) + 2*sqrt(2)))

Iter     Function value   Gradient norm 
     0     1.308356e+00     3.189312e+00
 * time: 2.002716064453125e-5
    10    -2.828427e+00     1.988244e-10
 * time: 0.00040984153747558594


Maximum: 2.828427 in 10 iterations
Diff: 0.0E+00


In [18]:
# Gradients provided, large scale, use limited memory BFGS
res = Optim.optimize(f, x0, Optim.LBFGS(), Optim.Options(show_trace = verbose, show_every = 10); autodiff = :forward)
@printf("\n\nMaximum: %f in %d iterations\n", -Optim.minimum(res), Optim.iterations(res))
@printf("Diff: %0.1E\n", abs(Optim.minimum(res) + 2*sqrt(2)))

Iter     Function value   Gradient norm 
     0     1.308356e+00     3.189312e+00
 * time: 1.1920928955078125e-5


Maximum: 2.828427 in 9 iterations
Diff: 4.4E-16


# Exercices

- Find a "nice" algebraic expression for the maximizer corresponding to the objective value $2 \sqrt{2}$
- Add support for the complex part, see if there is any difference (real vs. complex quantum mechanics!).
- Parameterize the quantum state as $\left| \gamma \right> = \cos \gamma \left| 01 \right> + \sin \gamma \left| 10 \right>$. Which state leads to the largest violation?

## Bonus open ended exercices

- Plot the objective value for the iteration number; either use the trace, a callback, or just stop the algorithm after $n$ steps.
- (Beware package incompatibilities) Parameterize $\cos \alpha, \sin \alpha$ as $x_1, x_2$ with $x_1^2 + x_2^2 = 1$ for each angle, and rewrite the above as a polynomial optimization problem. Solve with JuMP and the polynomial optimization extensions. Can you certify that $2 \sqrt(2)$ is a global maximum?